# Packing stability problem

In [1]:
from pandas import DataFrame, read_csv
import os
from os import listdir
from os.path import isfile, join
import math
import numpy as np
import random
import copy

In [16]:
file_list = [f for f in listdir(os.getcwd()) if isfile(f) and ('csv' in f)]
file_list

['0-out.csv',
 '1-out.csv',
 '2-out.csv',
 '3-out.csv',
 '4-out.csv',
 '5-out.csv',
 '6-out.csv',
 '7-out.csv',
 '8-out.csv',
 '9-out.csv']

In [18]:
def loadData(file_name):
    data = read_csv(os.getcwd()+'/'+file_name, sep=',',header = 1,index_col=False)
    return data
file_name = '0-out.csv'
data = loadData(file_name)
data.head(5)

,900001,0,0.1,0.2,600,400,200,1,12000,Unnamed: 9
0,900001,0,400,0,600,800,200,1,12000,NaN
1,900001,600,0,0,1200,400,200,1,12000,NaN
2,900001,600,400,0,1200,800,200,1,12000,NaN
3,900001,0,0,200,600,400,400,1,12000,NaN
4,900001,0,400,200,600,800,400,1,12000,NaN


In [ ]:
class Box():
    #конструктор создает коробку с SKU и начальными параметрами. Изначальная ротация = 3
    def __init__(self, SKU, length, width, height, weight, strength, aisle, caustic):
        self.SKU = SKU
        self.size = np.array([length,width,height])
        self.coords = self.size
        self.rotation = 3
        self.weight = weight
        self.strength = strength
        self.aisle = aisle
        self.caustic = caustic
        self.attPoint = np.array([-1,-1,-1])
        self.farCorner = np.array([-1,-1,-1])
        
    #private функция возвращает матрицу поворота 1-6
    def __getRotationMatrix(self, rotation):
        if rotation == 1:
            return np.array([[1,0,0],[0,0,1],[0,1,0]])
        elif rotation == 2:
            return np.array([[0,0,1],[1,0,0],[0,1,0]])
        elif rotation == 3:
            return np.array([[1,0,0],[0,1,0],[0,0,1]])
        elif rotation == 4:
            return np.array([[0,0,1],[0,1,0],[1,0,0]])
        elif rotation == 5:
            return np.array([[0,1,0],[1,0,0],[0,0,1]])
        elif rotation == 6:
            return np.array([[0,1,0],[0,0,1],[1,0,0]])
    
    #метод позволяющий повернуть коробку в положение 1-6 (3 положение по умолчанию)
    def setRotation(self, rotation):
        self.rotation = rotation
        self.coords = np.dot(self.size, self.__getRotationMatrix(rotation))
        self.farCorner = self.attPoint+self.coords
    
    #вернуть реальные координаты в свой СО
    def getCoords(self):
        return self.coords
    
    #вернуть SKU
    def getSKU(self):
        return self.SKU
    
    #вернуть длину (x по умолчанию)
    def getLength(self):
        return self.size[0]
    
    #вернуть ширину (y по умолчанию)
    def getWidth(self):
        return self.size[1]
    
    #вернуть высоту (z по умолчанию)
    def getHeight(self):
        return self.size[2]
    
    #вернуть свои исходные размеры
    def getSize(self):
        return list(self.size)
    
    #вернуть свое текущее положение
    def getRotation(self):
        return self.rotation
    
    def getWeigth(self):
        return self.weight
    
    def getStrength(self):
        return self.strength
    
    def getAisle(self):
        return self.aisle
    
    def getCaustic(self):
        return self.caustic
    
    def __repr__(self):
        return 'Box(SKU=%s, rot=%s, coords=%s, attPoint=%s, farCorner=%s)' % (self.SKU, self.rotation, self.coords,self.attPoint, self.farCorner)
    
    def displace(self, dcoords):
        self.attPoint = np.array(dcoords)
        self.farCorner = self.attPoint+self.coords
        
    def getAttPoints(self):
        p1 = self.attPoint+self.coords*[1,0,0]
        p2 = self.attPoint+self.coords*[0,1,0]
        p3 = self.attPoint+self.coords*[1,1,0]
        p4 = self.attPoint+self.coords*[0,0,1]
        x = list([list(p1),list(p2),list(p3),list(p4)])
        return x
    
    def getAttPoint(self):
        return list(self.attPoint)
    
    def getFarCorner(self):
        return list(self.farCorner)
    
    def getBottomCenterMass(self):
        return ((self.farCorner[0]+self.attPoint[0]) / 2.0 , (self.farCorner[1]+self.attPoint[1]) / 2.0, self.attPoint[2])
    
    def getMaxZ(self):
        return self.farCorner[2]
    
    def getVolume(self):
        return self.size[0]*self.size[1]*self.size[2]
    
    def getUpperPlane(self):
        return ((self.attPoint[0],self.attPoint[1], self.farCorner[2]), (self.farCorner[0],self.farCorner[1], self.farCorner[2]))
    
    def getAnswer(self):
        return [self.SKU, 
                self.attPoint[0], self.attPoint[1], self.attPoint[2], 
                self.farCorner[0], self.farCorner[1], self.farCorner[2],
                self.aisle,
                self.weight
               ]
    
    def __eq__(self, other):
        """Override the default Equals behavior"""
        if (self.getSKU() == other.getSKU() and
           self.getRotation() == other.getRotation() and
           self.getAttPoint() == other.getAttPoint() and
           self.getSize() == other.getSize()):
            return True
        else:
            return False
    
    def __ne__(self, other):
        if  not (self.getSKU() == other.getSKU() and
           self.getRotation() == other.getRotation() and
           self.getAttPoint() == other.getAttPoint() and
           self.getSize() == other.getSize()):
            return True
        else:
            return False
    
    def getPerimeter(self):
        return self.getLength() * 2 + self.getWidth() * 2

In [ ]:
def getCenterMassHeight(boxes):
    for i in range(1, len(boxes)):
        mass_total += boxes[i].getWeight()
        x_total += boxes[i].getLength() * boxes[i].getWeight()
        y_total += boxes[i].getWidth() * boxes[i].getWeight()
        z_total += boxes[i].getHeight() * boxes[i].getWeight()
    centerMass = (x_total / mass_total, y_total / mass_total, z_total / mass_total)
    return z_total / mass_total

In [ ]:
def perimeter_all(boxes):
    for i in range(1, len(boxes)):
        perimeter_all += boxes[i].getPerimeter()

In [ ]:
def perimeter_common(box1,box2):
    def getOverlap(a, b):
            return max(0, min(a[1], b[1]) - max(a[0], b[0]))

        def line_x(box1,box2):
            l_x_1 = (box1.attPoint[0], box1.farCorner[0])
            l_x_2 = (box2.attPoint[0], box2.farCorner[0])
            return getOverlap(l_x_1, l_x_2)

        def line_xy(box1,box2):
            l_x_1 = (box1.attPoint[0], box1.farCorner[0])
            l_x_2 = (box2.attPoint[0], box2.farCorner[0])
            l_y_1 = (box1.attPoint[1], box1.farCorner[1])
            l_y_2 = (box2.attPoint[1], box2.farCorner[1])
            return getOverlap(l_x_1, l_x_2) + getOverlap(l_y_1, l_y_2)

        def line_y(b1,b2):
            l_y_1 = (box1.attPoint[1], box1.farCorner[1])
            l_y_2 = (box2.attPoint[1], box2.farCorner[1])
            return getOverlap(l_y_1, l_y_2)
    return line_x(b1,b2) + line_y(b1,b2) + line_xy(b1,b2)

In [19]:
def stability_coeff():
    короче if CenterMassHeight < (условно) 50:
        elif  perimeter_all - perimeter_common > (условно) 50:
            return заебись (1)
        else return ооо всё упадёт, аккуратней


SyntaxError: invalid syntax (<ipython-input-19-38fa4e8921d4>, line 2)